In [ ]:
from datasets import load_dataset

# 正确的方式：即使是从本地加载，也要添加 trust_remote_code=True
dataset = load_dataset(
    "IWSLT/da2023",
)

print(dataset)

/home/hwt-wsl-ubt/codespace/toy-codes-hwt/encdec-from-nanogpt/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


DatasetDict({
    train: Dataset({
        features: ['lp', 'src', 'mt', 'ref', 'raw', 'annotators', 'domain', 'year', 'task', 'sys_id'],
        num_rows: 12480
    })
})


In [ ]:
print(dataset['train'][0])

{'lp': 'en-de', 'src': 'Then CEAS applies the generator to the four labeled documents independently and generates release notes for each class.', 'mt': 'Anschließend wendet C-E-A-S den Generator unabhängig voneinander auf die vier Etikettendokumente an und erzeugt für jede Klasse Freisetzungsnotizen.', 'ref': 'Dann wendet CEAS den Generator auf die vier markierten Dokumente unabhängig voneinander an und erstellt Versionshinweise für jede Klasse.', 'raw': 80, 'annotators': 1, 'domain': 'ACL', 'year': 2023, 'task': 'multilingual', 'sys_id': 2}


In [ ]:

# 创建一个迭代器，用于逐批提供文本数据
def batch_iterator(batch_size=1000):
    for i in range(0, len(dataset["train"]), batch_size):
        batch = dataset["train"][i : i + batch_size]
        # 从'src'和'ref'列中提取文本
        yield [text for text in batch["src"] if text is not None]
        yield [text for text in batch["ref"] if text is not None]

In [5]:
from tokenizers import Tokenizer
from tokenizers.models import BPE
from tokenizers.trainers import BpeTrainer
from tokenizers.pre_tokenizers import Whitespace

# 1. 初始化一个分词器
tokenizer = Tokenizer(BPE(unk_token="[UNK]"))

# 2. 设置前置分词器（pre-tokenizer），这里我们使用空格分词
tokenizer.pre_tokenizer = Whitespace()

# 3. 初始化一个训练器
# vocab_size: 词汇表大小
# special_tokens: 特殊标记
trainer = BpeTrainer(vocab_size=30000, special_tokens=["[UNK]", "[CLS]", "[SEP]", "[PAD]", "[MASK]"])

# 4. 训练分词器
tokenizer.train_from_iterator(batch_iterator(), trainer=trainer)

# 5. 保存训练好的分词器
tokenizer.save("iwslt-da2023-bpe-tokenizer.json")